In [ ]:
from pathlib import Path

import cv2
import matplotlib as plt
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
from scipy.ndimage import zoom

In [ ]:
# def resizeAndPad(img, size, padColor=0):

#     h, w = img.shape[:2]
#     sh, sw = size

#     # interpolation method
#     if h > sh or w > sw: # shrinking image
#         interp = cv2.INTER_AREA
#     else: # stretching image
#         interp = cv2.INTER_CUBIC

#     # aspect ratio of image
#     aspect = w/h 

#     # compute scaling and pad sizing
#     if aspect > 1: # horizontal image
#         new_w = sw
#         new_h = np.round(new_w/aspect).astype(int)
#         pad_vert = (sh-new_h)/2
#         pad_top, pad_bot = np.floor(pad_vert).astype(int), np.ceil(pad_vert).astype(int)
#         pad_left, pad_right = 0, 0
#     elif aspect < 1: # vertical image
#         new_h = sh
#         new_w = np.round(new_h*aspect).astype(int)
#         pad_horz = (sw-new_w)/2
#         pad_left, pad_right = np.floor(pad_horz).astype(int), np.ceil(pad_horz).astype(int)
#         pad_top, pad_bot = 0, 0
#     else: # square image
#         new_h, new_w = sh, sw
#         pad_left, pad_right, pad_top, pad_bot = 0, 0, 0, 0

#     # set pad color
#     if len(img.shape) is 3 and not isinstance(padColor, (list, tuple, np.ndarray)): # color image but only one color provided
#         padColor = [padColor]*3

#     # scale and pad
#     scaled_img = cv2.resize(img, (new_w, new_h), interpolation=interp)
#     scaled_img = cv2.copyMakeBorder(scaled_img, pad_top, pad_bot, pad_left, pad_right, borderType=cv2.BORDER_CONSTANT, value=padColor)

#     return scaled_img

In [ ]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized

In [ ]:
def getROI(filepath, size, asix, slice):

    mri_img = nib.load(filepath)
    n1_header = mri_img.header

    pixdim = mri_img.header['pixdim'][1:4]
    dim = mri_img.header['dim'][1:4]

    img_data = mri_img.get_fdata()

  

    new_mri_img = zoom(img_data, pixdim)
    r, a, s = new_mri_img.shape
    if 'T1' in filepath:
        if asix == 'r':
            img_normalized = cv2.normalize(new_mri_img[:, :, int(s / 2) - 1 + slice], None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            img_normalized = cv2.rotate(img_normalized, cv2.ROTATE_90_COUNTERCLOCKWISE)
        if asix == 'a':
            img_normalized = cv2.normalize(new_mri_img[int(r / 2) - 1 + slice, :, :], None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            img_normalized = cv2.rotate(img_normalized, cv2.ROTATE_180)
            img_normalized = cv2.flip(img_normalized, 1)
        if asix == 's':
            img_normalized = cv2.normalize(new_mri_img[:, int(a / 2) - 1 + slice, :], None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            img_normalized = cv2.flip(img_normalized, 1)
    else:
        if asix == 'r':
            img_normalized = cv2.normalize(new_mri_img[int(r / 2) - 1 + slice, :, :], None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            img_normalized = cv2.rotate(img_normalized, cv2.ROTATE_90_COUNTERCLOCKWISE)
            img_normalized = cv2.flip(img_normalized, 1)
        if asix == 'a':
            img_normalized = cv2.normalize(new_mri_img[:, int(a / 2) - 1 + slice, :], None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            img_normalized = cv2.rotate(img_normalized, cv2.ROTATE_90_COUNTERCLOCKWISE)
            img_normalized = cv2.flip(img_normalized, 1)
        if asix == 's':
            img_normalized = cv2.normalize(new_mri_img[:, :, int(s / 2) - 1 + slice], None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            img_normalized = cv2.rotate(img_normalized, cv2.ROTATE_90_COUNTERCLOCKWISE)
        
    img = cv2.merge([img_normalized, img_normalized, img_normalized])
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if 'DTI' not in filepath:
        img_bw = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1] #+ cv2.THRESH_OTSU
    else:
        img_bw = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1] #+ cv2.THRESH_OTSU

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4, 4))
    erosion = cv2.dilate(img_bw,kernel,iterations = 1)

    cnts = cv2.findContours(erosion, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    
    c = max(cnts, key = cv2.contourArea)
   
    x, y, w, h = cv2.boundingRect(c)
    #cv2.rectangle(img, (x, y), (x + w, y + h), color=(255, 0, 0), thickness=1)
    x_new = x - 3 if x - 3 > 0 else 0
    y_new = y - 5 if y - 5 > 0 else 0
    w_new = w + 3 if w + 3 < img.shape[0] else img.shape[0]
    h_new = h + 5 if h + 5 < img.shape[1] else img.shape[1]
    img_crop = img[y_new:y+h_new, x_new:x+w_new]
    if h > w:
        img_resize = image_resize(img_crop, height = size[1])
    else:
        img_resize = image_resize(img_crop, width = size[0])
    top_size, bottom_size, left_size, right_size = (50, 50, 50, 50)
    replicate = cv2.copyMakeBorder(img_resize, top_size, bottom_size, left_size, right_size, cv2.BORDER_REPLICATE)
    
    
    x = replicate.shape[1] / 2 - size[1]/2
    y = replicate.shape[0] / 2 - size[0]/2
    w = size[0]
    h = size[1]
    img_resize_crop = replicate[int(y):int(y+h), int(x):int(x+w)]
    #scaled_img = resizeAndPad(img_crop, size)
    #cv2.normalize(img_resize_crop, img_resize_crop, 0, 1, cv2.NORM_MINMAX)
    #img_resize_crop *= 255
    maxindex  = img_resize_crop.max()
    minindex = img_resize_crop.min()
    print('resize image max'+ str(maxindex) + 'min'+str(minindex))
    return img_resize_crop

In [ ]:

# scaled_img = getROI('./ixi_augmentation/T1/RandomMotion/RM-IXI002-Guys-0828-T1.nii.gz', (64,64), 's', 0)
# imgplot = plt.imshow(scaled_img)
# plt.show()

In [ ]:
import ruamel.yaml
yaml = ruamel.yaml.YAML()
yaml.representer.ignore_aliases = lambda *data: True

DefaultConfigPath = "Config.yaml"

with open(DefaultConfigPath, encoding="utf-8") as inyaml:
    yaml_data = yaml.load(inyaml)

In [ ]:
from pathlib import Path
import os
for key in yaml_data["modalities"]:
        folder = Path(yaml_data["output_slice_dir_name"], key)
        if not os.path.exists(folder): 
            os.makedirs(folder)
        dataset_dir = Path(yaml_data["output_augmentation_dir_name"])
        images_dir = dataset_dir / key #/ key1
        print(images_dir)
        for root,dirs,names in os.walk(images_dir):
            for filename in names:
                if filename.endswith('.nii.gz') and '._' not in filename:
                    for slice in yaml_data["slice"]:
                        slice_name = filename.rstrip('.nii.gz') + yaml_data["asix"] +'_' +str(slice) + '.png'
                        folder = Path(yaml_data["output_slice_dir_name"], key, filename.rstrip('.nii.gz'))
                        if not os.path.exists(folder): 
                            os.makedirs(folder)
                        scaled = getROI(os.path.join(root,filename), (64,64), yaml_data["asix"], slice)
                        cv2.imwrite(os.path.join(folder,slice_name), scaled)
                        #print(os.path.join(folder,slice_name))

In [ ]:
from pathlib import Path
import os
for key in yaml_data["modalities"]:
    for key1 in yaml_data["transform"]:
        folder = Path(yaml_data["output_slice_dir_name"], key, key1)
        if not os.path.exists(folder): 
            os.makedirs(folder)
        dataset_dir = Path(yaml_data["output_augmentation_dir_name"])
        images_dir = dataset_dir / key / key1

        for root,dirs,names in os.walk(images_dir):
            for filename in names:
                if filename.endswith('.nii.gz') and '._' not in filename:
                    for slice in yaml_data["slice"]:
                        slice_name = filename.rstrip('.nii.gz') + yaml_data["asix"] +'_' +str(slice) + '.png'
                        folder = Path(yaml_data["output_slice_dir_name"], key, key1, filename.rstrip('.nii.gz'))
                        if not os.path.exists(folder): 
                            os.makedirs(folder)
                        scaled = getROI(os.path.join(root,filename), (64,64), yaml_data["asix"], slice)
                        cv2.imwrite(os.path.join(folder,slice_name), scaled)
                    #print(os.path.join(root,filename)) 
        #image_paths = sorted(images_dir.glob('*.nii.gz'))
        #scaled_img = getROI(image_paths, (64,64), 's', 0)


In [ ]:
from pathlib import Path
import cv2
root = Path('/Users/yu/Documents/CT/')
size = (64,64)
for root,dirs,names in os.walk(root):
    for filename in names:
        if filename.endswith('.png'):
            
                img = cv2.imread(os.path.join(root,filename))
                
                h, w = img.shape[:2]
                if h > w:
                    img_resize = image_resize(img, height = size[1])
                else:
                    img_resize = image_resize(img, width = size[0])
                    print(os.path.join(root,'CT',filename))
                folder = os.path.join(root,'CT',filename).rstrip('.jpeg')
                if not os.path.exists(folder): 
                    os.makedirs(folder)
                top_size, bottom_size, left_size, right_size = (50, 50, 50, 50)
                replicate = cv2.copyMakeBorder(img_resize, top_size, bottom_size, left_size, right_size, cv2.BORDER_REPLICATE)
                
                
                x = replicate.shape[1] / 2 - size[1]/2
                y = replicate.shape[0] / 2 - size[0]/2
                w = size[0]
                h = size[1]
                img_resize_crop = replicate[int(y):int(y+h), int(x):int(x+w)]
                cv2.imwrite(os.path.join(folder,filename),img_resize_crop)

In [ ]:
from split import splits_2020 as splits

for i in range(len(splits['ixi_slice'])):
    known = splits['ixi_slice'][len(splits['ixi_slice'])-i-1]

In [ ]:
known

In [ ]:
splits['ixi_slice']

In [5]:
from torchvision.datasets import ImageFolder
class ixi_slice_Filter(ImageFolder):
    """Tiny_ImageNet Dataset.
    """

    def __Filter__(self, known):
        datas, targets = self.imgs, self.targets
        new_datas, new_targets = [], []
        for i in range(len(datas)):
            if datas[i][1] in known:
                new_item = (datas[i][0], known.index(datas[i][1]))
                new_datas.append(new_item)
                # new_targets.append(targets[i])
                new_targets.append(known.index(targets[i]))
        datas, targets = new_datas, new_targets
        self.samples, self.imgs, self.targets = datas, datas, targets

In [6]:
dataroot='./data/'
img_size=64
from torchvision import transforms
train_transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.RandomCrop(img_size, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
        ])
trainset = ixi_slice_Filter(os.path.join(
            dataroot, 'ixi_slice', 'ixi_slice', 'train'), train_transform)

print(trainset.class_to_idx)
print('All Train Data:', len(trainset))
trainset.__Filter__(known=[0, 1, 14, 15, 16])

{'CT': 0, 'MRA': 1, 'RandomBiasField_T1': 2, 'RandomBiasField_T2': 3, 'RandomElasticDeformation_T1': 4, 'RandomElasticDeformation_T2': 5, 'RandomGhosting_T1': 6, 'RandomGhosting_T2': 7, 'RandomMotion_T1': 8, 'RandomMotion_T2': 9, 'RandomNoise_T1': 10, 'RandomNoise_T2': 11, 'RandomSpike_T1': 12, 'RandomSpike_T2': 13, 'T1': 14, 'T2': 15, 'X-Ray': 16}
All Train Data: 6402


In [18]:
known=[0, 14, 1, 15, 16]
for i in known:
    for key, value in trainset.class_to_idx.items():
        if i == value:
            print(key)


CT
T1
MRA
T2
X-Ray


In [14]:

trainset = ImageFolder(os.path.join('./data', 'ixi_slice', 'ixi_slice', 'train'))

legendname = []
for i in known:
    for key, value in trainset.class_to_idx.items():
        if i == value:
            legendname.append(key)

In [15]:
legendname

['CT', 'RandomBiasField_T1', 'T1', 'RandomNoise_T2', 'X-Ray']

In [ ]:
import os
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from torchvision.datasets import ImageFolder
train_loader = torch.utils.data.DataLoader(
            trainset, batch_size=64, shuffle=True,
            num_workers=1, pin_memory=False,
        )

In [1]:
from datasets.osr_dataloader import ixi_slice_OSR

Data = ixi_slice_OSR(known=[0, 1, 14, 15, 16], dataroot='./data/ixi_slice/',
                             batch_size=64, img_size=64)
trainloader, testloader, outloader = Data.train_loader, Data.test_loader, Data.out_loader

/opt/anaconda3/envs/MA/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Selected Labels:  [0, 1, 14, 15, 16]
Selected unknown Labels:  [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
All Train Data: 6402
All Test Data: 837
Train:  2773 Test:  384 Out:  453
All Test:  837


In [3]:
Data.num_classes

5

In [2]:
if __name__=='__main__':
    for i, item in enumerate(trainloader):
        print('i:', i)
        data, label = item
        print('data:', data)
        print('label:', label)

i: 0
data: tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0314,  ..., 0.0314, 0.0314, 0.0314],
          ...,
          [0.0000, 0.0000, 0.0314,  ..., 0.0392, 0.0392, 0.0392],
          [0.0000, 0.0000, 0.0314,  ..., 0.0431, 0.0431, 0.0431],
          [0.0000, 0.0000, 0.0314,  ..., 0.0353, 0.0353, 0.0353]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0314,  ..., 0.0314, 0.0314, 0.0314],
          ...,
          [0.0000, 0.0000, 0.0314,  ..., 0.0392, 0.0392, 0.0392],
          [0.0000, 0.0000, 0.0314,  ..., 0.0431, 0.0431, 0.0431],
          [0.0000, 0.0000, 0.0314,  ..., 0.0353, 0.0353, 0.0353]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.03

KeyboardInterrupt: 